Changelog

* v5:
  * new dataset with longer-running fio benchmarks (`runtime_seconds=60`)
  * => CPU times shifted a little upwards, probably due to zio background work
  * played around with log scales for latency and pmem time, found it more confusing than useful though
    * => commented out

In [ ]:
import pandas as pd
import glob
import json
import dotted # https://pypi.org/project/dotted-notation/
import re
import matplotlib.pyplot as plt
import numpy as np

from pathlib import Path
import seaborn as sns
import lib.datasciencetoolbelt as dstools
from lib.resultstorage import ResultStorage

In [ ]:
dstools.setup({
    "seaborn_context": "talk",
    "savefig": {
        "enable": True,
        "dir": Path("./postprocess_results"),
    }
})
result_storage = ResultStorage(Path("./results"))

#%matplotlib qt
%matplotlib inline


In [ ]:
result_storage_prefix = "ncommitters_scalability__v5"

id_vars__dottedpath_and_shortname_and_type = [
    ('pmem_setup_data.interleaving', "interleaving", int),    
    ("storage_stack.config.module_args.zfs.zfs_zil_pmem_prb_ncommitters", "ncommitters", int),
    ("fio_config.numjobs", "numjobs", int),
]
id_vars = [p[1] for p in id_vars__dottedpath_and_shortname_and_type]

def extract_id_var_values(output_json):
    d = output_json
    id_var_values = {}
    for dp, sn, ty in id_vars__dottedpath_and_shortname_and_type: 
        v = dotted.get(d, dp)
        if not v:
            raise Exception(f"{d['file']}: dotted path {dp} not found")
        if sn in id_var_values:
            raise Exception(f"duplicate shortname {sn}")
        try:
            id_var_values[sn] = ty(v)
        except ValueError as e:
            raise Exception(f"cannot parse v={v!r}") from e
    return id_var_values

def get_fio_write_metrics(output_json):
    d = output_json
    jobs = dotted.get(d, "fio_jsonplus.jobs")
    assert len(jobs) == 1
    j0 = jobs[0]
    jw = jobs[0]["write"]
    return jw

def to_unified_dict(output_json):
    d = output_json
    
    try:
        jw = get_fio_write_metrics(output_json)

        return {
            **extract_id_var_values(output_json),

            # fio
            "w_iops_mean": jw["iops_mean"],
            "w_iops_stddev": jw["iops_stddev"],
            "w_lat_mean": dotted.get(jw, "lat_ns.mean"),
            "w_lat_stddev": dotted.get(jw, "lat_ns.stddev"),

            # kstats
            **d["zvol_stats"],
            **d["itxg_bypass_stats"],
            **d["zil_pmem_stats"],
            **d["zil_pmem_ringbuf_stats"],
            "bio_total": d["zvol_stats"]["submit_bio__zvol_write(with_taskq_if_enabled)"],
            "taskq_delay": dotted.get(d, 'zvol_stats.zvol_write__taskq_qdelay'),
            "assign_aquire": dotted.get(d, 'itxg_bypass_stats.assign__aquisition_total'),
            "assign_vtable": dotted.get(d, 'itxg_bypass_stats.assign__vtable'),
            "assign_total": dotted.get(d, 'itxg_bypass_stats.assign__total'),
            "commit_total": dotted.get(d, 'itxg_bypass_stats.commit__total'),
            "commit_aquire": dotted.get(d, 'itxg_bypass_stats.commit__aquire'),

            # cpu stats
            **{f"cpu_{comp}": val for comp, val in dotted.get(d, "cpu_time.allcpu").items()},
        }
    except Exception as e:
        import json
        print(json.dumps(output_json))
        raise 

In [ ]:
rows = [{**to_unified_dict(j)} for j in result_storage.iter_results(result_storage_prefix)]
df = pd.DataFrame.from_dict(rows)
df = df.set_index(id_vars, verify_integrity=True)
df

In [ ]:
#post-process cpu utilization
tmp = df.filter(regex="^cpu_.*", axis=1)
# display(tmp)
cpu_total = tmp.sum(axis=1)
df['cpu_not_idle'] = cpu_total - df.cpu_idle
# second socket was disabled => half of total cpu time is idle time
df['cpu_utilization'] = df.cpu_not_idle / (cpu_total - (cpu_total/2))

# FactorizedDataFrame

In [ ]:
import itertools

def filter_by_index_value(df, level, filter):
    """Return a new df that only contains rows whose MultiIndex column `level`'s value passes `filter`"""
    return df[df.index.get_level_values(level).map(filter)]

def remove_index_dimension(df, level, value):
    """Reduce dimensionality of a dataframe by filtering by and subsequently dropping one of its index levels.
    
    df is assumed to be a multi-indexed pd.DataFrame.
    First, filter the data frame so that we only keep rows whose index tuple has value `value` at level `level`.
    Now the resulting data frame only has a single value at the level.
    Thus remove that level from the index.
    Voila: dimensionality reduced.
    """
    df = df[df.index.get_level_values(level) == value]
    assert set(df.index.get_level_values(level)) == {value}
    df.index = df.index.droplevel(level)
    return df

def _test_remove_index_dimension():
    data = [{"favnum": n, "favletter": l, "id": id} for id, (n, l) in enumerate(itertools.product([23,42],["a", "b"]))]
    d = pd.DataFrame(data).set_index(["favnum", "favletter"])
    display(d)
    display(remove_index_dimension(d, "favnum", 23))
    display(remove_index_dimension(d, "favletter", "b"))
    
_test_remove_index_dimension()

In [ ]:
def level_values_sorted_unique(df, level):
    """Returns the sorted unique values of a DataFrame's multi-index at level `level`"""
    return sorted(list(set(df.index.get_level_values(level))))

class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self
        
class FactorizedDataFrameItem(AttrDict):
    @property
    def title(self):
        if self.fdf.row and self.fdf.col:
            return f"{self.fdf.row}={self.rv}|{self.fdf.col}={self.cv}"
        elif self.fdf.row:
            return f"{self.fdf.row}={self.rv}"
        elif self.fdf.col:
            return f"{self.fdf.col}={self.cv}"
        else:
            return ""
            
        
class FactorizedDataFrame:
    def __init__(self, data, row, col):
        self.data = data
        self.col = col
        self.row = row

        self.col_values = [None] if not self.col else level_values_sorted_unique(self.data, self.col)
        self.row_values = [None] if not self.row else level_values_sorted_unique(self.data, self.row)
        
    def iter_factorized(self):
        for ci, c in enumerate(self.col_values):
            for ri, r in enumerate(self.row_values):
                d = self.data.copy()
                if c:
                    d = remove_index_dimension(d, self.col, c)
                if r:
                    d = remove_index_dimension(d, self.row, r)
                # display(d)
            
                context = FactorizedDataFrameItem({
                    "fdf": self,
                    "d": d,
                    "ri": ri,
                    "rv": r,
                    "ci": ci,
                    "cv": c,
                    "is_last_row": ri == len(self.row_values)-1,
                    "is_last_col": ci == len(self.col_values)-1,
                })
                yield context
                

def factorplot(data=None, row=None, col=None, plot=None, subplots_kw={}):
    """Factorizez MultiIndex'ed DataFrame `data`, then invokes `plot` for each FactorizedDataFrameItem"""
    
    fdf = FactorizedDataFrame(data, row, col)
    
    subplots_kw = {
        "gridspec_kw": {'hspace': 1},
        **subplots_kw,
        "squeeze": False, # axes should always be two-dimensional
    }

    fig, axes = plt.subplots(len(fdf.row_values), len(fdf.col_values), **subplots_kw)

    for f in fdf.iter_factorized():
        ax = axes[f.ri, f.ci]
        ax.set_title(f.title)
        legend = f.ri == len(fdf.row_values)-1 and f.ci == len(fdf.col_values)-1
        plot(f, ax, legend)
        if legend:
            plt.legend(loc='lower left', bbox_to_anchor=(1,0.5))

# Committerslot Histogram

In [ ]:
tmp = df.copy()
bucketprefix = "prb_write__committerslothist_b_"
buckets = list(filter(lambda col: col.find(bucketprefix) == 0, tmp.columns))
rename = {col: col[len(bucketprefix):] for col in buckets}
df_cslot = tmp[buckets].copy()
df_cslot = df_cslot.rename(rename, axis=1)
df_cslot = df_cslot.rename_axis(columns='bucket')
df_cslot

In [ ]:
# ensure that other is zero
assert (df_cslot['other'] == 0).all()
# drop it
del df_cslot['other']

In [ ]:
tmp = df_cslot.copy()
tmp = pd.DataFrame(tmp.stack().rename('count').reset_index())
tmp = tmp.set_index(id_vars + ["bucket"])
df_cslot = tmp

In [ ]:
tmp = df_cslot.copy()
tmp = tmp.reset_index()
tmp['bucket'] = tmp.bucket.astype('int64')
tmp = tmp.set_index(id_vars + ["bucket"])
df_cslot = tmp

In [ ]:
tmp = df_cslot.copy()
tmp = tmp.reset_index()
tmp['weight'] = tmp.bucket.map(lambda v: v + 1)
tmp = tmp.set_index(id_vars + ["bucket"])
df_cslot = tmp

## Average Committer Slot

In [ ]:
countsum = df_cslot['count'].unstack('bucket').sum(axis=1)
countsum

In [ ]:
weightedcount = (df_cslot['count'] * df_cslot['weight']).unstack('bucket').sum(axis=1)
weightedcount

In [ ]:
avg_committer_slot = pd.DataFrame((weightedcount / countsum).rename('avg_committer_slot'))
avg_committer_slot

In [ ]:
df = df.merge(avg_committer_slot, left_index=True, right_index=True)

In [ ]:
ax = avg_committer_slot.unstack('ncommitters').plot(figsize=(10,5))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

## Committer Slot Distribution


In [ ]:
df_cslot['count'].unstack('bucket')

In [ ]:
tmp = df_cslot.copy()
tmp = tmp['count'].unstack('bucket')
# delete colums that only contain zeroes
# https://stackoverflow.com/questions/21164910/how-do-i-delete-a-column-that-contains-only-zeros-in-pandas
tmp = tmp.loc[:, (tmp != 0).any(axis=0)]

In [ ]:
tmp = tmp.div(tmp.sum(axis=1), axis=0)

In [ ]:
tmp = tmp.query("numjobs in [1,4,8,12,16,24]")
ncommitters_values = sorted(list(set(tmp.index.get_level_values('ncommitters'))))
print(ncommitters_values)
numjobs_values = sorted(list(set(tmp.index.get_level_values('numjobs'))))


def plot(f, ax, legend):
#     display(f.d)
    f.d.plot.bar(ax=ax, stacked=True, legend=False)
#     f.d.plot.area(ax=ax, legend=False)
    
    if not f.is_last_row:
        ax.set_xticklabels([])
        ax.set_xlabel("")
    
    
factorplot(tmp, col='interleaving', row='numjobs', plot=plot,
                subplots_kw={
                    "figsize": (15,  5 + len(numjobs_values)*1.2),
            #         "figsize": (10, (0.6666) * (5 + 3*1)),
                    "gridspec_kw": {
                        "hspace": 1,
                    },
                })

# for i in ncommitters_values:
#     tmp.query('ncommitters == @i').plot.area(figsize=(15,1.5), legend=False)

# CPU Time Spent Per IOP

In [ ]:
def cpu_iop_df():
    return df.copy()
#     return df.copy().query('ncommitters in [1,2,4,8,16,18]')

### CPU Utilization

In [ ]:
data = cpu_iop_df()

data = data[["cpu_utilization"]].unstack("ncommitters")
ax = data.plot(figsize=(12,5), ylim=(0, 1.1))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

###  IOPS

In [ ]:
data = cpu_iop_df()
# data = data.query('numjobs in [1,4,8,16] and ncommitters in [1,2,4,8,16]')

data = data[["w_iops_mean"]].unstack("ncommitters")
ax = data.plot(figsize=(12,5), ylim=(0,None))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

###  CPU Per IOP

In [ ]:
data = cpu_iop_df()

data['cpu_per_iop'] = data.cpu_not_idle / data.w_iops_mean

data = data[["cpu_per_iop"]].unstack("ncommitters")
ax = data.plot(figsize=(12,5), ylim=(0, None))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

### Combined

In [ ]:
data = cpu_iop_df()
# data = data.query('numjobs in [1,4,8,16] and ncommitters in [1,2,4,8,16]')
data = data.query('ncommitters in [1,2,3,4,8,12,24]')
data = data.query('numjobs <= 18')

data['cpu_per_iop'] = data.cpu_not_idle / data.w_iops_mean
data['pmem_time_per_iop'] = data.prb_write__pmem / data.w_iops_mean


data = data[[
    "w_iops_mean",
    "w_lat_mean",
    "cpu_per_iop",
#     "avg_committer_slot",
    "pmem_time_per_iop",
    "w_lat_stddev",
]]
data = data.rename_axis("metric", axis=1)
data = pd.DataFrame(data.stack().rename("metric_value"))
data = data.sort_index()
# display(data)


def plot(f, ax, legend):
#     display(f.d)

    data = f.d.copy().unstack("ncommitters")
    xticks = list(range(0, 19, 2))
    if f.rv == "prb_write__pmem":
         ax = data.plot(ax=ax, ylim=(0, 150_000_000_000), xticks=xticks, legend=False)       
    elif f.rv == "pmem_time_per_iop":
        ylim=(1e5, (1e6))
#         ax = data.plot(ax=ax, logy=True, ylim=ylim, yticks=np.arange(ylim[0], ylim[1], (10**6)), xticks=xticks, legend=False)        
#         ax = data.plot(ax=ax, logy=True, ylim=ylim, yticks=np.arange(1e5, 1e7, 1e5), xticks=xticks, legend=False)       
        ax = data.plot(ax=ax, ylim=(10_000, 600_000), xticks=xticks, logy=False, legend=False)        
    elif f.rv == 'avg_committer_slot':
        ax = data.plot(ax=ax, ylim=(0,10), yticks=range(0,10), xticks=xticks, legend=False)        
    elif f.rv == 'cpu_per_iop':
        ax = data.plot(ax=ax, ylim=(0,0.007), xticks=xticks, legend=False)
    elif f.rv == 'w_iops_mean':
        # data = data[["cpu_per_iop"]].unstack("ncommitters")
        ax = data.plot(ax=ax, ylim=(0, 900_000),xticks=xticks,  legend=False)
        # ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    elif f.rv == 'w_lat_mean':
        # data = data[["cpu_per_iop"]].unstack("ncommitters")
        ax = data.plot(ax=ax, ylim=(0, 50_000), xticks=xticks, legend=False)
#         ax = data.plot(ax=ax, logy=True, ylim=(8000, 200_000), yticks=[1e4, 2e4, 4e4, 8e4, 1e5, 2e5, 4e5, 8e5], xticks=xticks, legend=False)
#         ax = data.plot(ax=ax, logy=True, ylim=(8000, 200_000), yticks=[1e4, 2e4, 3e4, 4e4, 5e4, 6e4, 7e4, 8e4, 9e4, 1e5, 2e5, 3e5, 4e5], xticks=xticks, legend=False)
        # ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    elif f.rv == 'w_lat_stddev':
        ax = data.plot(ax=ax, ylim=(0, 50_000), xticks=xticks,  legend=False)
    else:
        display(f.d)
        raise Exception("unknown row")
        
    if legend:
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    if not f.is_last_row:
        ax.set_xticklabels([])
        ax.set_xlabel("")
    
    
factorplot(data, col='interleaving', row='metric', plot=plot,
                subplots_kw={
                    "figsize": (20,  25),
            #         "figsize": (10, (0.6666) * (5 + 3*1)),
                    "gridspec_kw": {
                        "hspace": 0.1,
                    },
                })
